In [1]:
{-# LANGUAGE OverloadedStrings #-}

import AdventOfCode
import qualified Data.Attoparsec.Char8 as A

input <- dayLines 8

In [2]:
data Instr = Instr
    { instrRegister  :: String
    , instrOperation :: Int
    , instrAmount    :: Int
    , instrCondition :: Cond
    } deriving Show

data Comparison = LessThan | LessThanEqual | Equal | GreaterThanEqual | GreaterThan | NotEqual deriving Show

data Cond = Cond
    { condRegister   :: String
    , condComparison :: Comparison
    , condValue      :: Int
    } deriving Show

In [3]:
import Data.ByteString.UTF8 (toString)
import qualified Data.ByteString as B

notSpace = (/= ' ')

convertComparison :: B.ByteString -> Comparison
convertComparison c = case c of
    "<"  -> LessThan
    "<=" -> LessThanEqual
    "==" -> Equal
    ">=" -> GreaterThanEqual
    ">"  -> GreaterThan
    "!=" -> NotEqual

parseCond = do
    register <- toString <$> A.takeWhile notSpace
    A.space
    comp <- convertComparison <$> A.takeWhile notSpace
    A.space
    n <- A.signed A.decimal
    return $ Cond register comp n

parseInstr = do
    register <- toString <$> A.takeWhile notSpace
    A.space
    op <- A.choice $ map A.string ["inc", "dec"]
    let operation = if op == "inc" then 1 else -1
    A.space
    number <- A.signed A.decimal
    A.string " if "
    cond <- parseCond
    return $ Instr register operation number cond


parsedInstructions =  map (either error id . A.parseOnly parseInstr) input

In [10]:
import qualified Data.Set as S
import Data.List
import qualified Data.Map.Strict as M

registers = nub $ concatMap (\i -> [instrRegister i, condRegister (instrCondition i)]) parsedInstructions

initialRegisters = M.fromList $ zip registers $ repeat 0

In [11]:
evaluateCondition :: M.Map String Int -> Cond -> Bool
evaluateCondition registers condition = let
    registerValue = registers M.! condRegister condition
    compValue     = condValue condition
    in case condComparison condition of
        LessThan         -> registerValue <  compValue
        LessThanEqual    -> registerValue <= compValue
        Equal            -> registerValue == compValue
        GreaterThanEqual -> registerValue >= compValue
        GreaterThan      -> registerValue >  compValue
        NotEqual         -> registerValue /= compValue

In [12]:
go :: M.Map String Int -> Instr -> M.Map String Int
go state current = if evaluateCondition state (instrCondition current)
    then M.adjust (+(instrOperation current * instrAmount current)) (instrRegister current) state
    else state

finished = foldl' go initialRegisters parsedInstructions

maximum $ M.elems finished

5215

In [14]:
states = concatMap M.elems $ scanl go initialRegisters parsedInstructions
maximum states

6419